In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/02/20 15:15:30 WARN Utils: Your hostname, isolde resolves to a loopback address: 127.0.1.1; using 192.168.0.188 instead (on interface wlp1s0)
23/02/20 15:15:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 15:15:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/20 15:15:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/20 15:15:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/20 15:15:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/02/20 15:15:32 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/02/20 15:15:32 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/02/20 15:15:32 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [5]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
from datetime import datetime

In [8]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [9]:
rows = rdd.take(10)
row = rows[0]

In [10]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88)

In [11]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [12]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [13]:
from collections import namedtuple

In [14]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [15]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [16]:
from pyspark.sql import types

In [17]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [18]:
df_result = rdd.filter(filter_outliers) \
   .map(prepare_for_grouping) \
   .reduceByKey(calculate_revenue) \
   .map(unwrap) \
   .toDF(result_schema)

In [21]:
df_result.write.parquet('tmp/green-revenue', mode="overwrite")

23/02/20 15:17:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [35]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']
duration_rdd = df_green \
    .select(columns) \
    .rdd

In [36]:
import pandas as pd

In [37]:
rows = duration_rdd.take(10)

In [38]:
df = pd.DataFrame(rows, columns=columns)

In [39]:
list(df.itertuples())

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-12 18:15:04'), PULocationID=41, DOLocationID=41, trip_distance=0.78),
 Pandas(Index=1, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-31 20:24:10'), PULocationID=173, DOLocationID=70, trip_distance=0.98),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-07 08:16:53'), PULocationID=74, DOLocationID=236, trip_distance=2.7),
 Pandas(Index=3, VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-15 14:47:15'), PULocationID=25, DOLocationID=66, trip_distance=0.8),
 Pandas(Index=4, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-31 10:08:00'), PULocationID=259, DOLocationID=51, trip_distance=2.33),
 Pandas(Index=5, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-18 17:46:45'), PULocationID=177, DOLocationID=188, trip_distance=2.62),
 Pandas(Index=6, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-17 20:08:44'), PULocationID=65, DOLocationID=97, trip_distance=1.13),
 Pandas(In

In [40]:
def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [44]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [46]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [47]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|3.9000000000000004|
|               4.9|
|              13.5|
|               4.0|
|             11.65|
|13.100000000000001|
|5.6499999999999995|
| 6.800000000000001|
|             55.75|
|               8.9|
|               5.0|
|             13.75|
|               5.5|
|             19.05|
|              9.25|
|              45.7|
|               5.2|
| 5.699999999999999|
|              5.75|
|4.6000000000000005|
+------------------+
only showing top 20 rows

